# Post Training Quantization an EfficientDet Object Detection Model

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/example_keras_effdet_lite0.ipynb)

## Overview

In this notebook, we'll demonstrate the post-training quantization using MCT for a pre-trained object detection model in Keras. Specifically, we'll integrate a post-processing custom layer from [sony-custom-layers](https://github.com/sony/custom_layers) into the model. This integration aligns with the imx500 target platform capabilities.

In this example we will use an existing pre-trained EfficientDet model taken from [efficientdet-pytorch](https://github.com/rwightman/efficientdet-pytorch). We will convert the model to a Keras functional model that includes the custom [PostProcess Layer](https://github.com/sony/custom_layers/blob/main/sony_custom_layers/keras/object_detection/ssd_post_process.py). Further, we will quantize the model using MCT post training quantization and evaluate the performance of the floating point model and the quantized model on the COCO dataset.

We'll use the [timm](https://github.com/huggingface/pytorch-image-models)'s data loader and evaluation capabilities used for the original pytorch pretrained model. The conversion to the Keras model will not be covered. You can go over the conversion [here](https://github.com/sony/model_optimization/tree/main/tutorials/resources/efficientdet)


## Setup

install and import relevant packages

In [ ]:
!pip install tensorflow
!pip install model-compression-toolkit
!pip install torch
!pip install torchvision
!pip install timm
!pip install effdet
!git clone https://github.com/sony/model_optimization/tree/main/tutorials/resources

In order to convert the PyTorch you'll need to use the conversion code in the [MCT tutorials folder](https://github.com/sony/model_optimization/tree/main/tutorials)

In [ ]:
from time import time
import torch
from timm.utils import AverageMeter
from effdet.config import get_efficientdet_config
from effdet import create_dataset, create_loader, create_evaluator
from effdet.data import resolve_input_config
import model_compression_toolkit as mct
from resources.efficientdet import EfficientDetKeras, TorchWrapper
from resources.utils import load_state_dict

### Init data loader and evaluation functions

In [ ]:
def get_coco_dataloader(batch_size=16, split='val', config=None):
    root = '/data/projects/swat/datasets_src/COCO'

    args = dict(interpolation='bilinear', mean=None, std=None, fill_color=None)
    dataset = create_dataset('coco', root, split)
    input_config = resolve_input_config(args, config)
    loader = create_loader(
        dataset,
        input_size=input_config['input_size'],
        batch_size=batch_size,
        use_prefetcher=True,
        interpolation=input_config['interpolation'],
        fill_color=input_config['fill_color'],
        mean=input_config['mean'],
        std=input_config['std'],
        num_workers=0,
        pin_mem=False,
    )
    evaluator = create_evaluator('coco', dataset, pred_yxyx=False)

    return loader, evaluator


def acc_eval(_model, batch_size=16, config=None):
    val_loader, evaluator = get_coco_dataloader(batch_size=batch_size, config=config)

    batch_time = AverageMeter()
    end = time()
    last_idx = len(val_loader) - 1
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            output = _model(input, img_info=target)

            evaluator.add_predictions(output, target)

            # measure elapsed time
            batch_time.update(time() - end)
            end = time()
            if i % 10 == 0 or i == last_idx:
                print(
                    f'Test: [{i:>4d}/{len(val_loader)}]  '
                    f'Time: {batch_time.val:.3f}s ({batch_time.avg:.3f}s, {input.size(0) / batch_time.avg:>7.2f}/s)  '
                )

    return evaluator.evaluate()

### Create the model and copy weights from pretrained PyTorch weights file

In [ ]:
model_name = 'tf_efficientdet_lite0'
config = get_efficientdet_config(model_name)

merged_outputs = True
use_custom_layer = True
pretrained_backbone = False
model = EfficientDetKeras(config,
                          pretrained_backbone=pretrained_backbone
                          ).get_model([*config.image_size] + [3],
                                      merge_outputs=merged_outputs,
                                      use_custom_layer=use_custom_layer)

state_dict = torch.hub.load_state_dict_from_url(config.url, progress=False,
                                                map_location='cpu')
state_dict_numpy = {k: v.numpy() for k, v in state_dict.items()}
load_state_dict(model, state_dict_numpy)

### Wrap model in a Torch Module so it can be evaluated with timm's evaluation code

In [ ]:
wrapped_model = TorchWrapper(model,
                             merged_outputs=merged_outputs,
                             used_custom_layer=use_custom_layer)

float_map = acc_eval(wrapped_model, batch_size=64, config=config)

## Quantize Keras model


In [ ]:
loader, _ = get_coco_dataloader(split='train', config=config)


def get_representative_dataset(n_iter):

    def representative_dataset():
        ds_iter = iter(loader)
        for _ in range(n_iter):
            t = next(ds_iter)[0]
            yield [t.detach().cpu().numpy().transpose((0, 2, 3, 1))]

    return representative_dataset


quant_model, _ = mct.ptq.keras_post_training_quantization_experimental(model,
                                                                       get_representative_dataset(20))

### Wrap quantized model in a Torch Module so it can be evaluated with timm's evaluation code

In [ ]:
wrapped_model = TorchWrapper(quant_model,
                             merged_outputs=merged_outputs,
                             used_custom_layer=use_custom_layer)

quant_map = acc_eval(wrapped_model, batch_size=64, config=config)

print(f' ===>> Float mAP = {100*float_map:2.3f}, Quantized model mAP = {100*quant_map:2.3f}')